# Notebook: Compare LLM and Human Annotations

## Packages

In [1]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import numpy as np
import json
import sys
import os

In [2]:
sys.path.append(os.path.abspath('../07 train models/'))
from TASD.evaluation import calculate_metrics_for_examples
import constants

## Settings

In [3]:
LLMS = ["Llama70B", "GPT-3"]
FEW_SHOT_CONDITIONS = ["random", "fixed"]
AC_POLARITY_COMBINATIONS = [cat+"_"+polarity for cat in constants.ASPECT_CATEGORIES for polarity in constants.POLARITIES]

In [4]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}", "Llama70B": "\\textbf{Llama-2-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

## Code

### Load Data

#### Human Annotations

In [5]:
with open(f"annotation_datasets/annotated_synth_dataset.json", 'r') as json_file:
    human_annotations = json.load(json_file)    

#### Load Synthetic 

In [6]:
llm_annotations = []

for llm in LLMS:
    for fs in ["random", "fixed"]:
       for split_id in range(6):
           with open(f"../07 train models/synth/{llm}/{fs}/split_{split_id}.json", 'r') as json_file:
              synthetic_data_split = json.load(json_file)
              for example in  synthetic_data_split:
                  llm_annotations.append(example)    

In [7]:
llm_annotations_aspects = [([{"aspect_category": tag["label"], "aspect_polarity": tag["polarity"],
                              "aspect_term": tag["text"] if tag["text"] != 'NULL' else None, "start": tag["start"], "end": tag["end"]} for tag in example["tags"]], example["id"]) for example in llm_annotations]
human_annotations_aspects = [([{"aspect_category": tag["label"], "aspect_polarity": tag["polarity"], "aspect_term": tag["text"]
                                if tag["text"] != 'NULL' else None, "start": tag["start"], "end": tag["end"]} for tag in example["tags"]], example["id"], example["model"], example["few_shot_condtion"]) for example in human_annotations]

In [8]:
dataset = {}

In [9]:
def get_example_with_id(id, dataset):
    return [example for example in dataset if example[1] == id][0][0]

for llm in LLMS:
    dataset[llm] = {}
    for fs in FEW_SHOT_CONDITIONS:
        dataset[llm][fs] = {}
        human_annotations_aspects_ids = [example[1] for example in human_annotations_aspects if example[2] == llm and example[3] == fs]
        human_annotations_samples = [example[0] for example in human_annotations_aspects if example[2] == llm and example[3] == fs]
        llm_annotations_samples = [get_example_with_id(id, llm_annotations_aspects) for id in human_annotations_aspects_ids]

        dataset[llm][fs]["human_annotation"] = human_annotations_samples
        dataset[llm][fs]["llm_annotation"] = llm_annotations_samples

### Analyse Quality

#### Aspect Term Detection

In [10]:
def calculate_tp_tn_fp_fn_aspect_term(pred, label):
    pred_set = set(
        f"{range['start']}_{range['end']}" for range in pred)
    label_set = set(
        f"{range['start']}_{range['end']}" for range in label)

    tp_set = pred_set & label_set
    tp = len(tp_set)

    fp_set = pred_set - tp_set
    fp = len(fp_set)

    fn_set = label_set - tp_set
    fn = len(fn_set)

    return tp, 0, fp, fn


In [11]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["human_annotation"]]

        tp_total = tn_total = fp_total = fn_total = 0
        for i in range(len(human_annotations_aspect_terms)):
            tp, tn, fp, fn = calculate_tp_tn_fp_fn_aspect_term(
                llm_annotations_aspect_terms[i], human_annotations_aspect_terms[i])
            tp_total += tp
            tn_total += tn
            fp_total += fp
            fn_total += fn

        # Calculate metrics
        accuracy = (tp_total + tn_total) / (tp_total + tn_total + fp_total +
                                            fn_total) if (tp_total + tn_total + fp_total + fn_total) > 0 else 0
        precision = tp_total / \
            (tp_total + fp_total) if (tp_total + fp_total) > 0 else 0
        recall = tp_total / \
            (tp_total + fn_total) if (tp_total + fn_total) > 0 else 0

        f1 = 2 * tp_total / (2 * tp_total + fn_total + fp_total)

        llm_print = "\\multirow{2}{*}{" + LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        fs_print = ENCODE_CONDITION[fs]
        
        print(llm_print, "&", fs_print,
              "&", "{:.3f}".format(accuracy),
              "&", "{:.3f}".format(f1),
              "&", "{:.3f}".format(precision),
              "&", "{:.3f}".format(recall), "\\\\")
    print("\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{500}} & 0.708 & 0.829 & 0.868 & 0.793 \\
 & \textbf{LRS\textsubscript{25}} & 0.698 & 0.822 & 0.882 & 0.770 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{500}} & 0.579 & 0.733 & 0.932 & 0.604 \\
 & \textbf{LRS\textsubscript{25}} & 0.603 & 0.752 & 0.916 & 0.638 \\
\hline


#### Aspect Category

In [12]:
def category_list_to_label(cat_list):
    return [1 if cat in cat_list else 0 for cat in constants.ASPECT_CATEGORIES]

In [13]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        accuracy = accuracy_score(
            human_annotations_aspect_categories, llm_annotations_aspect_categories)
        f1_micro = f1_score(human_annotations_aspect_categories,
                            llm_annotations_aspect_categories, average='micro')
        f1_macro = f1_score(human_annotations_aspect_categories,
                            llm_annotations_aspect_categories, average='macro')

        llm_print = "\\multirow{2}{*}{" + LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        print(llm_print, "&", ENCODE_CONDITION[fs],
              "&", "{:.3f}".format(accuracy),
              "&", "{:.3f}".format(f1_micro),
              "&", "{:.3f}".format(f1_macro), "\\\\")
    print("\\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{500}} & 0.721 & 0.818 & 0.809 \\
 & \textbf{LRS\textsubscript{25}} & 0.635 & 0.770 & 0.761 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{500}} & 0.909 & 0.953 & 0.953 \\
 & \textbf{LRS\textsubscript{25}} & 0.919 & 0.959 & 0.958 \\
\hline


#### Aspect Category (performance for each Aspect Category)

In [14]:
def category_list_to_label(cat_list):
    return [1 if cat in cat_list else 0 for cat in constants.ASPECT_CATEGORIES]


for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_categories = [category_list_to_label(
            [tag["aspect_category"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        for ac_idx, aspect_category in enumerate(constants.ASPECT_CATEGORIES):
            idx = constants.ASPECT_CATEGORIES.index(aspect_category)

            tp = sum((llm_annotations_aspect_categories[i][idx] == 1) and (
                human_annotations_aspect_categories[i][idx] == 1) for i in range(len(llm_annotations_aspect_categories)))
            fp = sum((llm_annotations_aspect_categories[i][idx] == 1) and (
                human_annotations_aspect_categories[i][idx] == 0) for i in range(len(llm_annotations_aspect_categories)))
            fn = sum((llm_annotations_aspect_categories[i][idx] == 0) and (
                human_annotations_aspect_categories[i][idx] == 1) for i in range(len(llm_annotations_aspect_categories)))

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            accuracy = accuracy_score([example[idx] for example in human_annotations_aspect_categories], [
                                      example[idx] for example in llm_annotations_aspect_categories])

            f1 = 2 * tp / (2 * tp + fn + fp)

            n_samples_in_class = sum(
                example[idx] == 1 for example in human_annotations_aspect_categories)

            llm_print = "\\multirow{10}{*}{" + \
                LLMS_ENCODED[llm] + "}" if fs_idx == 0 and ac_idx == 0 else ""
            fs_print = "\\multirow{5}{*}{" + \
                ENCODE_CONDITION[fs] + "}" if ac_idx == 0 else ""

            print(llm_print, "&", fs_print,
                  "&", "\\texttt{"+aspect_category+"}",
                  "&", "{:.3f}".format(f1),
                  "&", "{:.3f}".format(accuracy),
                  "&", "{:.3f}".format(precision),
                  "&", "{:.3f}".format(recall), "\\\\")

        print("\\arrayrulecolor{gray}\cline{2-7}\\arrayrulecolor{black}")

    print("\\hline")

\multirow{10}{*}{\textbf{Llama-2-70B}} & \multirow{5}{*}{\textbf{LRS\textsubscript{500}}} & \texttt{GENERAL-IMPRESSION} & 0.607 & 0.842 & 0.468 & 0.862 \\
 &  & \texttt{FOOD} & 0.776 & 0.870 & 0.871 & 0.700 \\
 &  & \texttt{SERVICE} & 0.872 & 0.929 & 0.904 & 0.842 \\
 &  & \texttt{AMBIENCE} & 0.880 & 0.937 & 0.875 & 0.885 \\
 &  & \texttt{PRICE} & 0.911 & 0.955 & 0.865 & 0.963 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{5}{*}{\textbf{LRS\textsubscript{25}}} & \texttt{GENERAL-IMPRESSION} & 0.568 & 0.811 & 0.456 & 0.755 \\
 &  & \texttt{FOOD} & 0.758 & 0.850 & 0.872 & 0.671 \\
 &  & \texttt{SERVICE} & 0.834 & 0.905 & 0.810 & 0.859 \\
 &  & \texttt{AMBIENCE} & 0.786 & 0.884 & 0.820 & 0.754 \\
 &  & \texttt{PRICE} & 0.861 & 0.920 & 0.804 & 0.927 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
\hline
\multirow{10}{*}{\textbf{GPT-3.5-turbo}} & \multirow{5}{*}{\textbf{LRS\textsubscript{500}}} & \texttt{GENERAL-IMPRESSION} & 0.940 & 0.971 & 0.896 & 0.988 \\

#### Aspect Category + Sentiment Polarity

In [15]:
def category_polarity_list_to_label(cat_pol_list):
    return [1 if ac_pol in cat_pol_list else 0 for ac_pol in AC_POLARITY_COMBINATIONS]

In [16]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"]+"_"+tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"]+"_"+tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        accuracy = accuracy_score(
            human_annotations_ac_pol, llm_annotations_ac_pol)
        f1_micro = f1_score(human_annotations_ac_pol,
                            llm_annotations_ac_pol, average='micro')
        f1_macro = f1_score(human_annotations_ac_pol,
                            llm_annotations_ac_pol, average='macro')

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        print(llm_print,
              "&", ENCODE_CONDITION[fs],
              "&", "{:.3f}".format(accuracy),
              "&", "{:.3f}".format(f1_micro),
              "&", "{:.3f}".format(f1_macro), "\\\\")
    print("\\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{500}} & 0.494 & 0.573 & 0.538 \\
 & \textbf{LRS\textsubscript{25}} & 0.421 & 0.532 & 0.492 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{500}} & 0.795 & 0.846 & 0.835 \\
 & \textbf{LRS\textsubscript{25}} & 0.771 & 0.854 & 0.845 \\
\hline


In [17]:
def category_polarity_list_to_label(cat_polarity_list):
    return [1 if cat_polarity in cat_polarity_list else 0 for cat_polarity in AC_POLARITY_COMBINATIONS]


for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"] + "_" + tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_ac_pol = [category_polarity_list_to_label(
            [tag["aspect_category"] + "_" + tag["aspect_polarity"] for tag in example]) for example in dataset[llm][fs]["human_annotation"]]

        for ac_pol_idx, ac_pol_combination in enumerate(AC_POLARITY_COMBINATIONS):
            idx = AC_POLARITY_COMBINATIONS.index(ac_pol_combination)

            tp = sum((llm_annotations_ac_pol[i][idx] == 1) and (
                human_annotations_ac_pol[i][idx] == 1) for i in range(len(llm_annotations_ac_pol)))
            fp = sum((llm_annotations_ac_pol[i][idx] == 1) and (
                human_annotations_ac_pol[i][idx] == 0) for i in range(len(llm_annotations_ac_pol)))
            fn = sum((llm_annotations_ac_pol[i][idx] == 0) and (
                human_annotations_ac_pol[i][idx] == 1) for i in range(len(llm_annotations_ac_pol)))

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            accuracy = accuracy_score([example[idx] for example in human_annotations_ac_pol], [
                                      example[idx] for example in llm_annotations_ac_pol])
            f1 = 2 * tp / (2 * tp + fn + fp)

            n_samples_in_class = sum(
                example[idx] == 1 for example in human_annotations_ac_pol)

            aspect_category, sentiment_polarity = ac_pol_combination.split("_")

            llm_print = "\\multirow{30}{*}{" + \
                LLMS_ENCODED[llm] + \
                "}" if fs_idx == 0 and ac_pol_idx == 0 else ""
            fs_print = "\\multirow{15}{*}{" + \
                ENCODE_CONDITION[fs] + "}" if ac_pol_idx == 0 else ""
            ac_print = "\\multirow{3}{*}{" + "\\texttt{" + \
                aspect_category+"}" + "}" if ac_pol_idx % 3 == 0 else ""

            print(llm_print,
                  "&", fs_print,
                  "&", ac_print,
                  "&", "\\texttt{"+sentiment_polarity+"}",
                  "&", "{:.3f}".format(accuracy),
                  "&", "{:.3f}".format(f1),
                  "&", "{:.3f}".format(precision),
                  "&", "{:.3f}".format(recall), "\\\\")

            if ac_pol_idx % 3 == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{3-8}\\arrayrulecolor{black}")

        print("\\cline{2-8}")
    print("\\hline")

\multirow{30}{*}{\textbf{Llama-2-70B}} & \multirow{15}{*}{\textbf{LRS\textsubscript{500}}} & \multirow{3}{*}{\texttt{GENERAL-IMPRESSION}} & \texttt{POSITIVE} & 0.929 & 0.577 & 0.552 & 0.604 \\
 &  &  & \texttt{NEUTRAL} & 0.902 & 0.235 & 0.139 & 0.769 \\
 &  &  & \texttt{NEGATIVE} & 0.928 & 0.415 & 0.315 & 0.607 \\
\arrayrulecolor{gray}\cline{3-8}\arrayrulecolor{black}
 &  & \multirow{3}{*}{\texttt{FOOD}} & \texttt{POSITIVE} & 0.887 & 0.586 & 0.841 & 0.449 \\
 &  &  & \texttt{NEUTRAL} & 0.923 & 0.474 & 0.371 & 0.657 \\
 &  &  & \texttt{NEGATIVE} & 0.950 & 0.713 & 0.788 & 0.651 \\
\arrayrulecolor{gray}\cline{3-8}\arrayrulecolor{black}
 &  & \multirow{3}{*}{\texttt{SERVICE}} & \texttt{POSITIVE} & 0.902 & 0.660 & 0.851 & 0.538 \\
 &  &  & \texttt{NEUTRAL} & 0.922 & 0.316 & 0.190 & 0.923 \\
 &  &  & \texttt{NEGATIVE} & 0.958 & 0.750 & 0.824 & 0.689 \\
\arrayrulecolor{gray}\cline{3-8}\arrayrulecolor{black}
 &  & \multirow{3}{*}{\texttt{AMBIENCE}} & \texttt{POSITIVE} & 0.916 & 0.650 & 0.867 &

#### Aspect Term + Sentiment Polarity

In [18]:
def calculate_tp_tn_fp_fn_e2e(pred, label):
    pred_set = set(
        f"{range['start']}_{range['end']}_{range['polarity']}" for range in pred)
    label_set = set(
        f"{range['start']}_{range['end']}_{range['polarity']}" for range in label)

    tp_set = pred_set & label_set
    tp = len(tp_set)

    fp_set = pred_set - tp_set
    fp = len(fp_set)

    fn_set = label_set - tp_set
    fn = len(fn_set)

    return tp, 0, fp, fn

In [19]:
def calculate_tp_tn_fp_fn_e2e_total(pred_total, label_total):
    tp_total = tn_total = fp_total = fn_total = 0
    for i in range(len(label_total)):
        tp, tn, fp, fn = calculate_tp_tn_fp_fn_e2e(
            pred_total[i], label_total[i])
        tp_total += tp
        tn_total += tn
        fp_total += fp
        fn_total += fn
    return tp_total, tn_total, fp_total, fn_total

In [20]:
def f1_macro_e2e(llm_annotations_aspect_terms, human_annotations_aspect_terms):
    f1_scores = []
    for pol in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
        llm_annotations_class = [[tag for tag in example if tag["polarity"] == pol]
                                 for example in llm_annotations_aspect_terms]
        human_annotations_class = [[tag for tag in example if tag["polarity"] == pol]
                                   for example in human_annotations_aspect_terms]
        
        tp_total, tn_total, fp_total, fn_total = calculate_tp_tn_fp_fn_e2e_total(llm_annotations_class, human_annotations_class)
        f1_scores.append(2 * tp_total / (2 * tp_total + fn_total + fp_total))
    return np.mean(f1_scores)

In [21]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        llm_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"], "polarity": tag["aspect_polarity"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["llm_annotation"]]
        human_annotations_aspect_terms = [
            [{"start": tag["start"], "end": tag["end"], "polarity": tag["aspect_polarity"]} for tag in example if tag["aspect_term"] is not None] for example in dataset[llm][fs]["human_annotation"]]

        tp_total, tn_total, fp_total, fn_total = calculate_tp_tn_fp_fn_e2e_total(
            llm_annotations_aspect_terms, human_annotations_aspect_terms)

        # Calculate metrics
        accuracy = (tp_total + tn_total) / (tp_total + tn_total + fp_total +
                                            fn_total) if (tp_total + tn_total + fp_total + fn_total) > 0 else 0
        precision = tp_total / \
            (tp_total + fp_total) if (tp_total + fp_total) > 0 else 0
        recall = tp_total / \
            (tp_total + fn_total) if (tp_total + fn_total) > 0 else 0

        f1_micro = 2 * tp_total / (2 * tp_total + fn_total + fp_total)

        f1_macro = f1_macro_e2e(
            llm_annotations_aspect_terms, human_annotations_aspect_terms)

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""

        print(llm_print, "&", ENCODE_CONDITION[fs],
              "&", "{:.3f}".format(accuracy),
              "&", "{:.3f}".format(f1_micro),
              "&", "{:.3f}".format(f1_macro), "\\\\")
    print("\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{500}} & 0.409 & 0.580 & 0.549 \\
 & \textbf{LRS\textsubscript{25}} & 0.403 & 0.574 & 0.538 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{500}} & 0.509 & 0.674 & 0.669 \\
 & \textbf{LRS\textsubscript{25}} & 0.517 & 0.681 & 0.675 \\
\hline


#### Aspect Term + Aspect Category + Sentiment Polarity

In [22]:
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, fs in enumerate(FEW_SHOT_CONDITIONS):
        metrics_triplets = calculate_metrics_for_examples(
            dataset[llm][fs]["human_annotation"], dataset[llm][fs]["llm_annotation"])

        llm_print = "\\multirow{2}{*}{" + \
            LLMS_ENCODED[llm] + "}" if fs_idx == 0 else ""
        print(llm_print,
              "&", ENCODE_CONDITION[fs],
              "&", "{:.3f}".format(metrics_triplets["accuracy"]),
              "&", "{:.3f}".format(metrics_triplets["f1"]),
              "&", "{:.3f}".format(metrics_triplets["precision"]),
              "&", "{:.3f}".format(metrics_triplets["recall"]), "\\\\")

    print("\\hline")

\multirow{2}{*}{\textbf{Llama-2-70B}} & \textbf{LRS\textsubscript{500}} & 0.307 & 0.469 & 0.443 & 0.499 \\
 & \textbf{LRS\textsubscript{25}} & 0.272 & 0.427 & 0.405 & 0.452 \\
\hline
\multirow{2}{*}{\textbf{GPT-3.5-turbo}} & \textbf{LRS\textsubscript{500}} & 0.395 & 0.566 & 0.562 & 0.571 \\
 & \textbf{LRS\textsubscript{25}} & 0.399 & 0.570 & 0.560 & 0.581 \\
\hline
